In [ ]:
#Data preprocessing
!python3 MT-Preparation/filtering/filter.py Tatoeba.en-tr.tr Tatoeba.en-tr.en tr en

In [ ]:
!python3 MT-Preparation/train_dev_split/train_dev_test_split.py 10000 10000 Tatoeba.en-tr.tr-filtered.tr Tatoeba.en-tr.en-filtered.en

In [3]:
fp = open("Data-train-src.txt", "w")
with open("Tatoeba.en-tr.tr-filtered.tr.train", 'r') as file:
    for i in range(300000):
        fp.write(file.readline())

fp.close()

fp = open("Data-train-src.txt", "a")
with open("kk_train_shuffled.txt-filtered.kk", 'r') as file:
    for i in range(300000):
        fp.write(file.readline())

fp.close()

In [4]:
fp = open("Data-train-tgt.txt", "w")
with open("Tatoeba.en-tr.en-filtered.en.train", 'r') as file:
    for i in range(300000):
        fp.write(file.readline())

fp.close()

fp = open("Data-train-tgt.txt", "a")
with open("en_train_shuffled.txt-filtered.en", 'r') as file:
    for i in range(300000):
        fp.write(file.readline())

fp.close()

In [ ]:
# Kazakh and Turkish shared vocabulary
!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab kk_tr_shared_vocab Data-train-src.txt

In [ ]:
#shared vocabulary English side
!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab en_shared_vocab Data-train-tgt.txt

In [8]:
# Kk-En -> Tr-En (Tatoeba)
!onmt-main --model kk-tr-en-shared.py --config data.yml --auto_config train --with_eval --num_gpus 2

2024-12-01 15:51:49.425203: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-01 15:51:50.214137: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-01 15:51:50.214204: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-01 15:51:50.214212: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [2]:
# Kk-En -> Tr-En (Tatoeba)
!onmt-main --model kk-tr-en-shared.py --config kk-tr-en-2.yml --auto_config --checkpoint_path KK-TR-EN-Shared-vocab/ckpt-100000 train --with_eval --num_gpus 2

2024-12-02 10:41:29.858199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-02 10:41:30.642218: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-02 10:41:30.642283: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-02 10:41:30.642291: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [3]:
!CUDA_VISIBLE_DEVICES=0 onmt-main --config data.yml --auto_config --checkpoint_path KK-TR-EN-Shared-vocab-2/ckpt-100000 infer --features_file Tatoeba_tokens_test_shared --predictions_file output_kk_en_tr_shared_vocab.txt

2024-12-03 09:44:54.066331: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 09:44:54.865685: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-03 09:44:54.865759: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-03 09:44:54.865781: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [ ]:
!python3 MT-Preparation/subwording/3-desubword.py en_shared_vocab.model output_kk_en_tr_shared_vocab.txt

In [1]:
# BLEU and chrF scores
!python3 compute-bleu.py Tatoeba.en-tr.en-filtered.en.test output_kk_en_tr_shared_vocab.txt.desubword

Reference first sentence: I won't stay there very long.
Translated first sentence: I won't stay there very long .
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
BLEU:  BLEU = 60.31 80.7/65.0/54.5/46.3 (BP = 1.000 ratio = 1.012 hyp_len = 78525 ref_len = 77587)
CHRF:  chrF2 = 74.08


In [6]:
# Average METEOR score (Ortalama METEOR Puanı)
import nltk
from nltk.translate.meteor_score import meteor_score

def read_and_tokenize_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return [nltk.word_tokenize(line.strip()) for line in lines]

def calculate_meteor(reference_file, hypothesis_file):
    references = read_and_tokenize_file(reference_file)
    hypotheses = read_and_tokenize_file(hypothesis_file)
    
    if len(references) != len(hypotheses):
        raise ValueError("Dosyaların satır sayıları eşleşmiyor")

    total_meteor_score = 0.0
    for ref, hyp in zip(references, hypotheses):
        total_meteor_score += meteor_score([ref], hyp)

    average_meteor_score = total_meteor_score / len(references)
    return average_meteor_score

reference_file = 'Tatoeba.en-tr.en-filtered.en.test'
hypothesis_file = 'output_kk_en_tr_shared_vocab.txt.desubword'

score = calculate_meteor(reference_file, hypothesis_file)
print(f"Ortalama METEOR Puanı: {score}") #Average METEOR score

Ortalama METEOR Puanı: 0.8145330897292876


In [7]:
# Tr-En (Tatoeba) -> Kk-En
!onmt-main --model kk-tr-en-shared.py --config tr-en-shared-vocab.yml --auto_config train --with_eval --num_gpus 2

2024-12-03 09:51:18.128381: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 09:51:18.931154: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-03 09:51:18.931225: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-03 09:51:18.931233: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [8]:
# Tr-En (Tatoeba) -> Kk-En
!onmt-main --model kk-tr-en-shared.py --config Tr-Kk-En_shared_vocab.yml --auto_config --checkpoint_path TR-EN-Shared-vocab/ckpt-100000 train --with_eval --num_gpus 2

2024-12-04 10:02:44.683064: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-04 10:02:45.478635: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-04 10:02:45.478703: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-04 10:02:45.478712: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [9]:
!CUDA_VISIBLE_DEVICES=0 onmt-main --config Tr-Kk-En_shared_vocab.yml --auto_config --checkpoint_path TR-KK-EN-Shared-vocab/ckpt-100000 infer --features_file KK_tokens_test_shared --predictions_file output_tr_kk_en_shared_vocab.txt

2024-12-05 09:19:05.616199: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-05 09:19:06.385567: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-05 09:19:06.385633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-05 09:19:06.385640: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [ ]:
!python3 MT-Preparation/subwording/3-desubword.py en_shared_vocab.model output_tr_kk_en_shared_vocab.txt

In [2]:
# BLEU and chrF scores
!python3 compute-bleu.py en_test_shuffled.txt-filtered.en output_tr_kk_en_shared_vocab.txt.desubword

Reference first sentence: In the developed world, this figure is 35 25%
Translated first sentence: In the developed countries of the world , this figure is 35 25%
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
BLEU:  BLEU = 51.49 72.0/55.8/45.7/38.3 (BP = 1.000 ratio = 1.120 hyp_len = 464044 ref_len = 414303)
CHRF:  chrF2 = 76.25


In [12]:
# Average METEOR score (Ortalama METEOR Puanı)
import nltk
from nltk.translate.meteor_score import meteor_score

def read_and_tokenize_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return [nltk.word_tokenize(line.strip()) for line in lines]

def calculate_meteor(reference_file, hypothesis_file):
    references = read_and_tokenize_file(reference_file)
    hypotheses = read_and_tokenize_file(hypothesis_file)
    
    if len(references) != len(hypotheses):
        raise ValueError("Dosyaların satır sayıları eşleşmiyor")

    total_meteor_score = 0.0
    for ref, hyp in zip(references, hypotheses):
        total_meteor_score += meteor_score([ref], hyp)

    average_meteor_score = total_meteor_score / len(references)
    return average_meteor_score

reference_file = 'en_test_shuffled.txt-filtered.en'
hypothesis_file = 'output_tr_kk_en_shared_vocab.txt.desubword'

score = calculate_meteor(reference_file, hypothesis_file)
print(f"Ortalama METEOR Puanı: {score}") #Average METEOR score

Ortalama METEOR Puanı: 0.7583990764796427
